<a href="https://www.kaggle.com/code/marwantosolve/mnist-phase3?scriptVersionId=214356923" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Install the essential libraries
!pip install cvxopt
!pip install idx2numpy

# Import the essential libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 76.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-py3-none-any.whl size=7904 sha256=7924213541496baa3c0b73e7a47005241a6b0325857f4babddfb3a180476c16a
  Stored in directory: /root/.cache/pip/wheels/e0/f4/e7/643fc5f932ec2ff92997f43f007660feb23f948aa8486f1107
Successfully built idx2numpy
/kaggle/input/mnist-dataset/train-images.idx3-ubyte
/kaggle/input/mnist-dataset/t10k-labels.idx1-ubyte
/kaggle/input/mnist-dataset/t10k-images.idx3-ubyte
/kaggle/input/mnist-dataset/train-labels.idx1-ubyte
/kaggle/input/mnist-dataset/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte
/kaggle/input/mnist-dataset/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte
/kaggle/input/mnist-dataset/train-labels-idx1-ubyte/train-labels-idx1-ubyte
/kaggle/input/mnist-dataset/train-images-idx3-ubyte/train-images-idx3-ubyte


**1. Data Preproccessing**

In [2]:
# Import the essential libraries
import idx2numpy

# Load MNIST dataset     
X_train = '/kaggle/input/mnist-dataset/train-images.idx3-ubyte'
y_train = '/kaggle/input/mnist-dataset/train-labels.idx1-ubyte'
X_test = '/kaggle/input/mnist-dataset/t10k-images.idx3-ubyte'
y_test = '/kaggle/input/mnist-dataset/t10k-labels.idx1-ubyte'

X_train = idx2numpy.convert_from_file(X_train)
y_train = idx2numpy.convert_from_file(y_train)
X_test = idx2numpy.convert_from_file(X_test)
y_test = idx2numpy.convert_from_file(y_test)

# Flatten the images
X_train = X_train.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)

# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

print("Data Preprocessing Applied Successfully ✓")

Data Preprocessing Applied Successfully ✓


**2. From Scratch SVM Classifier**

In [3]:
# Import the essential libraries to implement SVM using QP
from cvxopt import matrix, solvers

def train_binary_svm(X, y, C=1.0):
    """
    Train a binary SVM using quadratic programming.
    """
    n_samples, n_features = X.shape
    K = np.dot(X, X.T)  # Linear kernel
    P = matrix(np.outer(y, y) * K)
    q = matrix(-np.ones(n_samples))
    G = matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
    h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))
    A = matrix(y, (1, n_samples), 'd')
    b = matrix(0.0)
    
    # Solve QP problem
    solvers.options['show_progress'] = False
    solution = solvers.qp(P, q, G, h, A, b)
    alpha = np.ravel(solution['x'])
    
    # Compute weights and bias
    w = np.sum(alpha[:, None] * y[:, None] * X, axis=0)
    sv = alpha > 1e-5  # Support vectors
    b = np.mean(y[sv] - np.dot(X[sv], w))
    return w, b


def train_ovo_svm_in_batches(X, y, batch_size=2000, C=1.0):
    """
    Train One-vs-One SVM in mini-batches for all class pairs.
    """
    classes = np.unique(y)
    classifiers = {}
    
    for i, class1 in enumerate(classes):
        for j, class2 in enumerate(classes):
            if j <= i:
                continue  # Avoid duplicate pairs
            
            # Filter data for the two classes
            indices = (y == class1) | (y == class2)
            X_pair, y_pair = X[indices], y[indices]
            
            # Ensure NumPy arrays for batch processing
            X_pair = np.array(X_pair)
            y_pair = np.array(y_pair)
            
            # Convert labels to +1 and -1 for binary classification
            y_pair = np.where(y_pair == class1, -1, 1)
            
            # Train in batches
            n_samples = len(y_pair)
            indices_all = np.arange(n_samples)
            np.random.shuffle(indices_all)
            
            print(f"Training SVM for classes {class1} vs {class2}...")
            classifiers[(class1, class2)] = []
            for start in range(0, n_samples, batch_size):
                end = min(start + batch_size, n_samples)
                batch_indices = indices_all[start:end]
                X_batch, y_batch = X_pair[batch_indices], y_pair[batch_indices]
                
                w, b = train_binary_svm(X_batch, y_batch, C)
                classifiers[(class1, class2)].append((w, b))
    
    return classifiers

def predict_ovo_svm_in_batches(X, classifiers):
    """
    Predict the class of each sample using One-vs-One SVM with batch-trained classifiers.
    """
    n_samples = X.shape[0]
    votes = np.zeros((n_samples, 10))  # One vote counter for each class (10 total)
    
    for (class1, class2), models in classifiers.items():
        for w, b in models:
            # Decision function for this pair
            decision = np.dot(X, w) + b
            predictions = np.where(decision >= 0, class2, class1)
            
            # Increment votes for each predicted class
            for i, pred in enumerate(predictions):
                votes[i, pred] += 1
    
    # Final prediction is the class with the most votes
    return np.argmax(votes, axis=1)

#################################################

# Train SVM in batches using One-vs-One strategy
batch_size = 2000
C = 1.0
classifiers = train_ovo_svm_in_batches(X_train, y_train, batch_size=batch_size, C=C)

# Predict on test data
y_pred = predict_ovo_svm_in_batches(X_test, classifiers)

#####################################################

# Printing Confusion Matrix and Calculating Accuracy

# Import the essential libraries
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Evaluate the results
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(confusion_matrix(y_test, y_pred))
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Training SVM for classes 0 vs 1...
Training SVM for classes 0 vs 2...
Training SVM for classes 0 vs 3...
Training SVM for classes 0 vs 4...
Training SVM for classes 0 vs 5...
Training SVM for classes 0 vs 6...
Training SVM for classes 0 vs 7...
Training SVM for classes 0 vs 8...
Training SVM for classes 0 vs 9...
Training SVM for classes 1 vs 2...
Training SVM for classes 1 vs 3...
Training SVM for classes 1 vs 4...
Training SVM for classes 1 vs 5...
Training SVM for classes 1 vs 6...
Training SVM for classes 1 vs 7...
Training SVM for classes 1 vs 8...
Training SVM for classes 1 vs 9...
Training SVM for classes 2 vs 3...
Training SVM for classes 2 vs 4...
Training SVM for classes 2 vs 5...
Training SVM for classes 2 vs 6...
Training SVM for classes 2 vs 7...
Training SVM for classes 2 vs 8...
Training SVM for classes 2 vs 9...
Training SVM for classes 3 vs 4...
Training SVM for classes 3 vs 5...
Training SVM for classes 3 vs 6...
Training SVM for classes 3 vs 7...
Training SVM for cla

**2. Built-In SVM Classifier**

In [4]:
# Import the Built-In SVM Classifier
from sklearn.svm import SVC

# Train an SVM classifier
svm_model = SVC(kernel='linear')  # Using a linear kernel, but you can try others like 'rbf'
svm_model.fit(X_train, y_train)

# Make predictions
y_pred = svm_model.predict(X_test)

# Printing Confusion Matrix and Calculating Accuracy

# Import the essential libraries
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Evaluate the results
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(confusion_matrix(y_test, y_pred))
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

[[ 957    0    4    1    1    6    9    1    0    1]
 [   0 1122    3    2    0    1    2    1    4    0]
 [   8    6  967   11    3    3    7    8   17    2]
 [   4    3   16  947    1   16    0    9   12    2]
 [   1    1   10    1  942    2    4    2    3   16]
 [  10    4    3   36    6  803   13    1   14    2]
 [   9    2   13    1    5   16  910    1    1    0]
 [   1    8   21   10    8    1    0  957    3   19]
 [   8    4    6   25    7   26    6    7  877    8]
 [   7    7    2   11   33    4    0   18    5  922]]
Accuracy: 0.94
Precision: 0.94
Recall: 0.94
F1 Score: 0.94


**3. Built-In KNN Classifier**

In [5]:
# Import the Built-In KNN Classifier
from sklearn.neighbors import KNeighborsClassifier

# Train a KNN classifier
knn_model = KNeighborsClassifier(n_neighbors=3)  # Using k=3 as an example
knn_model.fit(X_train, y_train)

# Make predictions
y_pred = knn_model.predict(X_test)

# Printing Confusion Matrix and Calculating Accuracy

# Import the essential libraries
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Evaluate the results
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(confusion_matrix(y_test, y_pred))
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

[[ 974    1    1    0    0    1    2    1    0    0]
 [   0 1133    2    0    0    0    0    0    0    0]
 [  10    9  996    2    0    0    0   13    2    0]
 [   0    2    4  976    1   13    1    7    3    3]
 [   1    6    0    0  950    0    4    2    0   19]
 [   6    1    0   11    2  859    5    1    3    4]
 [   5    3    0    0    3    3  944    0    0    0]
 [   0   21    5    0    1    0    0  991    0   10]
 [   8    2    4   16    8   11    3    4  914    4]
 [   4    5    2    8    9    2    1    8    2  968]]
Accuracy: 0.97
Precision: 0.97
Recall: 0.97
F1 Score: 0.97


**4. Built-In NN Classifier**

In [6]:
# Import the Built-In MLP Classifier
from sklearn.neural_network import MLPClassifier

# Train an MLP classifier
nn_model = MLPClassifier(hidden_layer_sizes=(200,100,50), max_iter=300, random_state=42)  # One hidden layer with 100 neurons
nn_model.fit(X_train, y_train)

# Make predictions
y_pred = nn_model.predict(X_test)

# Printing Confusion Matrix and Calculating Metrics

# Import the essential libraries
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Evaluate the results
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(confusion_matrix(y_test, y_pred))
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

[[ 969    1    1    0    2    0    3    1    1    2]
 [   1 1126    2    1    0    0    2    1    2    0]
 [   4    1 1013    1    2    0    2    4    5    0]
 [   0    0    6  982    0    7    0    6    4    5]
 [   1    0    3    0  958    0    3    2    1   14]
 [   2    0    0    7    0  868    4    2    6    3]
 [   3    1    1    1    3    4  945    0    0    0]
 [   1    2    4    0    0    0    1 1013    2    5]
 [   3    0    2    5    1    5    0    3  951    4]
 [   2    2    0    2    6    2    1    4    3  987]]
Accuracy: 0.98
Precision: 0.98
Recall: 0.98
F1 Score: 0.98


**5. Built-In Random Forest Classifier**

In [7]:
# Import the Built-In Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

# Train a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Using 100 trees
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Printing Confusion Matrix and Calculating Metrics

# Import the essential libraries
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Evaluate the results
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(confusion_matrix(y_test, y_pred))
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

[[ 971    0    0    0    0    2    3    1    3    0]
 [   0 1127    2    2    0    1    2    0    1    0]
 [   6    0 1002    5    3    0    3    8    5    0]
 [   1    0    9  972    0    9    0    9    8    2]
 [   1    0    0    0  955    0    5    1    4   16]
 [   5    1    1    9    2  860    5    2    5    2]
 [   7    3    0    0    3    3  937    0    5    0]
 [   1    4   20    2    0    0    0  989    2   10]
 [   4    0    6    7    5    5    5    4  930    8]
 [   7    6    2   12   12    1    0    4    4  961]]
Accuracy: 0.97
Precision: 0.97
Recall: 0.97
F1 Score: 0.97
